In [253]:
import pandas as pd 
import os 
import numpy as np 
import pickle 

In [254]:
os.listdir('data_collection/data')

['for_exploration.ipynb',
 'art_topic_distribution.csv',
 'user_tweet_pair.csv',
 '.DS_Store',
 'user_tweet_topic.csv',
 'art_info_w_topic.csv',
 'linksWIP.csv',
 'art_info1.csv',
 'unq_links_2.pkl',
 'unique_user_topic_distribution.csv',
 'unq_links_1.pkl',
 'unq_links_1_2.pkl',
 'unq_links.pkl',
 'user_tweet.gexf',
 'nytposts_readyforarticle.csv',
 'linksFinal.csv',
 '.ipynb_checkpoints',
 'user_topic.gexf',
 'unq_links_2_2.pkl',
 'links.pkl',
 'unique_users.csv',
 'user_topic100.gexf']

In [3]:
users = pd.read_csv('data_collection/data/unique_users.csv')
users.head()

,Unnamed: 0,user_id,screen_name,location,retweet_list,retweet_count
0,0,55651793,['JSullivanIII'],"['Queens, NYC']","[1097372619848986625, 1096157615225212933, 109...",18
1,1,191972027,['ojessen'],['Kiel'],"[1097372619848986625, 1096696739128578048, 109...",8
2,2,22681390,['leehlawrence'],[''],"[1097372619848986625, 1097368941842837506, 109...",4
3,3,897359310308622336,['enaliyuddin'],"['Yogyakarta, Indonesia']","[1097372619848986625, 1093800374614585344]",2
4,4,4141824748,['eleftheria20464'],[''],"[1097372619848986625, 1097364848890007552, 109...",72


In [128]:
user_tweet = pd.read_csv('data_collection/data/user_tweet_pair.csv')
user_tweet.head()

,Unnamed: 0,user_id,screen_name,location,tweet_id
0,0,55651793,JSullivanIII,"Queens, NYC",1097372619848986625
1,1,191972027,ojessen,Kiel,1097372619848986625
2,2,22681390,leehlawrence,NaN,1097372619848986625
3,3,897359310308622336,enaliyuddin,"Yogyakarta, Indonesia",1097372619848986625
4,4,4141824748,eleftheria20464,NaN,1097372619848986625


### Adding weight to individual users 
For each tweet,  If t = total retweets

Then each of the twitter users who retweeted an article is assigned a weight of  1/t for that particulr tweet. 

In [129]:
#grouping by tweet_id
df = user_tweet.groupby('tweet_id').count()
df.reset_index(inplace=True)

In [75]:
user_tweet[user_tweet.tweet_id == 1086277077685301249].shape

(89, 5)

In [130]:
# Adding weights to column
df['weight'] = [1/x for x in df.user_id]

In [131]:
df.head()

,tweet_id,Unnamed: 0,user_id,screen_name,location,weight
0,1086277077685301249,89,89,89,68,0.011236
1,1086284595207778305,92,92,92,59,0.010870
2,1086292173996339200,75,75,75,49,0.013333
3,1086295933300674562,86,86,86,64,0.011628
4,1086299729565270016,91,91,91,67,0.010989


In [132]:
#checking to see the number of unique articles is the same
df.shape

(2434, 6)

In [133]:
df1 = pd.DataFrame()

In [134]:
df1['tweet_id'] = df.tweet_id
df1['weight'] = df.weight
df1['retweet_count'] = df.user_id

In [135]:
df1.shape

(2434, 3)

In [136]:
#merging with user_tweet to get all data 
df2 = pd.merge(user_tweet, df1, how = 'outer', on = ['tweet_id'])

In [137]:
df2.shape

(157747, 7)

In [138]:
df2.isna().any()

Unnamed: 0       False
user_id          False
screen_name      False
location          True
tweet_id         False
weight           False
retweet_count    False
dtype: bool

In [139]:
df2.head()

,Unnamed: 0,user_id,screen_name,location,tweet_id,weight,retweet_count
0,0,55651793,JSullivanIII,"Queens, NYC",1097372619848986625,0.027027,37
1,1,191972027,ojessen,Kiel,1097372619848986625,0.027027,37
2,2,22681390,leehlawrence,NaN,1097372619848986625,0.027027,37
3,3,897359310308622336,enaliyuddin,"Yogyakarta, Indonesia",1097372619848986625,0.027027,37
4,4,4141824748,eleftheria20464,NaN,1097372619848986625,0.027027,37


In [51]:
#calculating total weight for each unique user. 
#Total weight is the sum of weights from each tweet retweeted by user
D = {}
for i in range(df2.shape[0]): 
    user_id = df2.user_id[i]
    weight = df2.weight[i]
    if user_id not in D: 
        D[user_id] = 0
        D[user_id] += weight
    else: 
        D[user_id] += weight

In [52]:
len(D.keys()), users.shape

(76913, (76913, 7))

In [24]:
D[55651793]

0.4074595016247618

In [53]:
df3 = pd.DataFrame()
df3['user_id'] = D.keys()
df3['total_weight'] = D.values()
df3.head()

,user_id,total_weight
0,55651793,0.280275
1,191972027,0.137273
2,22681390,0.060897
3,897359310308622336,0.039527
4,4141824748,1.111009


In [54]:
users = users.merge(df3, how = 'inner', on = ['user_id'])

In [55]:
users.head()

,Unnamed: 0,user_id,screen_name,location,retweet_list,retweet_count,total_weight_x,total_weight_y
0,0,55651793,['JSullivanIII'],"['Queens, NYC']","[1097372619848986625, 1096157615225212933, 109...",18,0.407460,0.280275
1,1,191972027,['ojessen'],['Kiel'],"[1097372619848986625, 1096696739128578048, 109...",8,0.196842,0.137273
2,2,22681390,['leehlawrence'],[''],"[1097372619848986625, 1097368941842837506, 109...",4,0.088011,0.060897
3,3,897359310308622336,['enaliyuddin'],"['Yogyakarta, Indonesia']","[1097372619848986625, 1093800374614585344]",2,0.061277,0.039527
4,4,4141824748,['eleftheria20464'],[''],"[1097372619848986625, 1097364848890007552, 109...",72,1.714770,1.111009


In [60]:
df3.shape, users.shape

((76913, 2), (76913, 7))

In [66]:
users.head()

,user_id,screen_name,location,retweet_list,retweet_count,total_weight
0,55651793,['JSullivanIII'],"['Queens, NYC']","[1097372619848986625, 1096157615225212933, 109...",18,0.280275
1,191972027,['ojessen'],['Kiel'],"[1097372619848986625, 1096696739128578048, 109...",8,0.137273
2,22681390,['leehlawrence'],[''],"[1097372619848986625, 1097368941842837506, 109...",4,0.060897
3,897359310308622336,['enaliyuddin'],"['Yogyakarta, Indonesia']","[1097372619848986625, 1093800374614585344]",2,0.039527
4,4141824748,['eleftheria20464'],[''],"[1097372619848986625, 1097364848890007552, 109...",72,1.111009


In [67]:
# users.to_csv('data_collection/data/unique_users.csv')

### BUILD LINKS
1. We find a list of unique tweets 
2. For each tweet we have get the list of  retweeters 
3. We create all possible pair of users from the retweeter-list for that tweet
4. For each pair we give a weight of 1/(total retweeters)


In [140]:
print('df2 shape:', df2.shape)
df2.head()

df2 shape: (157747, 7)


,Unnamed: 0,user_id,screen_name,location,tweet_id,weight,retweet_count
0,0,55651793,JSullivanIII,"Queens, NYC",1097372619848986625,0.027027,37
1,1,191972027,ojessen,Kiel,1097372619848986625,0.027027,37
2,2,22681390,leehlawrence,NaN,1097372619848986625,0.027027,37
3,3,897359310308622336,enaliyuddin,"Yogyakarta, Indonesia",1097372619848986625,0.027027,37
4,4,4141824748,eleftheria20464,NaN,1097372619848986625,0.027027,37


In [141]:
#Unique tweet ids 
tweets = df2.tweet_id.unique()
len(tweets)

2434

In [231]:
#finding user pairs for each tweet and giving them id 
import pickle 
documents = []
count = 0
for tweet_id in tweets: 
    count += 1
    if count%200 == 0: 
        print(count)
        pickle.dump(documents,open('data_collection/data/links.pkl','wb'))
    mini_df = df2[df2['tweet_id'] == tweet_id]
    users = mini_df['user_id']
    for user1 in users[:-1]: 
        for user2 in users[1:]:
                D = {'tweet_id' : tweet_id, 
                     'user_1': user1, 
                     'user_2': user2, 
                     'weight': mini_df.weight.unique()[0]}
                documents.append(D)
        users = users[1:]
# pickle.dump(documents,open('data_collection/data/links.pkl','wb'))

200
400
600
800
1000
1200
1400
1600
1800
2000
2200
2400


In [232]:
# links = pickle.load(open('data_collection/data/links.pkl','rb'))
links = documents

In [233]:
links[1]

{'tweet_id': 1097372619848986625,
 'user_1': 55651793,
 'user_2': 22681390,
 'weight': 0.02702702702702703}

In [234]:
u = set([link['tweet_id'] for link in links])
len(u)

2434

In [235]:
len(links)

5788759

In [236]:
links_df = pd.DataFrame(links)
links_df.head()

,tweet_id,user_1,user_2,weight
0,1097372619848986625,55651793,191972027,0.027027
1,1097372619848986625,55651793,22681390,0.027027
2,1097372619848986625,55651793,897359310308622336,0.027027
3,1097372619848986625,55651793,4141824748,0.027027
4,1097372619848986625,55651793,2458778106,0.027027


In [237]:
links_df.shape

(5788759, 4)

In [238]:
len(links_df.tweet_id.unique())

2434

In [239]:
t = list(zip(links_df.user_1, links_df.user_2, links_df.index))

In [240]:
listy = []
for element in t: 
    listy.append(sorted(element))

In [241]:
links_df['u_1'] = [link[1] for link in listy]
links_df['u-2'] = [link[2] for link in listy]
links_df['u1_u2'] = [(str(link[1]) + str(link[2])) for link in listy]

In [242]:
links_df.head()

,tweet_id,user_1,user_2,weight,u_1,u-2,u1_u2
0,1097372619848986625,55651793,191972027,0.027027,55651793,191972027,55651793191972027
1,1097372619848986625,55651793,22681390,0.027027,22681390,55651793,2268139055651793
2,1097372619848986625,55651793,897359310308622336,0.027027,55651793,897359310308622336,55651793897359310308622336
3,1097372619848986625,55651793,4141824748,0.027027,55651793,4141824748,556517934141824748
4,1097372619848986625,55651793,2458778106,0.027027,55651793,2458778106,556517932458778106


In [243]:
links_df['duplicate'] = links_df.duplicated(subset=['u1_u2'], keep=False)

In [244]:
links_df.head()

,tweet_id,user_1,user_2,weight,u_1,u-2,u1_u2,duplicate
0,1097372619848986625,55651793,191972027,0.027027,55651793,191972027,55651793191972027,False
1,1097372619848986625,55651793,22681390,0.027027,22681390,55651793,2268139055651793,False
2,1097372619848986625,55651793,897359310308622336,0.027027,55651793,897359310308622336,55651793897359310308622336,False
3,1097372619848986625,55651793,4141824748,0.027027,55651793,4141824748,556517934141824748,False
4,1097372619848986625,55651793,2458778106,0.027027,55651793,2458778106,556517932458778106,False


In [246]:
links_df[links_df['duplicate']==True].shape

(591729, 8)

In [247]:
unq_pairs = links_df.u1_u2.unique()

In [166]:
unq_pairs[0]

'55651793191972027'

In [221]:
pair = '556517932458778106'
links_df[links_df['u1_u2'] == pair]

,tweet_id,user_1,user_2,weight,u_1,u-2,u1_u2,duplicate
4,1097372619848986625,55651793,2458778106,0.027027,55651793,2458778106,556517932458778106,True
180,1097372619848986625,2458778106,55651793,0.027027,55651793,2458778106,556517932458778106,True


In [170]:
links_df

0.02702702702702703

In [252]:
# links_df.to_csv('data_collection/data/linksWIP.csv')

### Merge links 

Each pair of users can have one or more tweets in common. 
Meaning links_df now has multiple links with the same pair but for different tweets. 
We now proceed to have a list of unique links for each unique pairs by summing up the weights for the different tweets. 
The links now represent how much commonality in news interests of each unique user pair 

In [255]:
links_df = pd.read_csv('data_collection/data/linksWIP.csv')

In [257]:
#Finding all links ('u1_u2') with multiple rows

duplicates = pd.DataFrame()
duplicates['u1_u2'] = links_df[links_df.duplicate == True]['u1_u2']
duplicates['weight'] = links_df[links_df.duplicate == True]['weight']
duplicates['tweet_id'] = links_df[links_df.duplicate == True]['tweet_id'].astype('str')
print(duplicates.shape)
duplicates.head()

(482853, 3)


,u1_u2,weight,tweet_id
18,556517933987328893,0.027027,1097372619848986625
30,4480016755651793,0.027027,1097372619848986625
35,556517932464156014,0.027027,1097372619848986625
40,191972027798666202511855616,0.027027,1097372619848986625
45,1919720271853538666,0.027027,1097372619848986625


In [258]:
# summing up the weights for all tweets 
duplicates1 = duplicates.groupby('u1_u2').sum()
print(duplicates1.shape)
duplicates1.head()

(169778, 1)


,weight
u1_u2,
10000182439380131841019510117027508224,0.024625
10000182439380131841092608411106267136,0.024625
10000182439380131841093857190715248640,0.024625
10000345566265917451086211707591118848,0.102870
10001559493053931521058540237264220165,0.027778


In [259]:
duplicates = duplicates.merge(duplicates1, how = 'inner', on = ['u1_u2'])
print(duplicates.shape)
duplicates.drop('weight_x', axis = 1, inplace = True)
duplicates.rename(columns = {'weight_y': 'weight'}, inplace = True)
duplicates.head()


(482853, 4)


,u1_u2,tweet_id,weight
0,556517933987328893,1097372619848986625,0.192412
1,556517933987328893,1096157615225212933,0.192412
2,556517933987328893,1095549389190377472,0.192412
3,556517933987328893,1095204615304417282,0.192412
4,556517933987328893,1094910173888606208,0.192412


In [260]:
unique_links  = list(duplicates.u1_u2.unique())

In [ ]:
#Making list of tweets for each link (link is a user to user pair 
#and the tweets are the common tweets that both users retweeted)
from tqdm import tqdm
import time 
tweets = []

for link in tqdm(unique_links): 
    df = duplicates[duplicates.u1_u2 == link]
    tweets.append(list(df.tweet_id))

In [ ]:
unique = pd.DataFrame()
unique['u1_u2'] = unique_links
unique['tweets'] = tweets
print(unique.shape)
unique.head(2)

In [ ]:
unique = unique.merge(duplicates1, how = 'inner', on = ['u1_u2'])

In [ ]:
unique.to_csv('data_collection/data/linksFinal.csv')

In [ ]:
u = unique.merge(duplicates, how = 'left', on = ['u1_u2']).drop_duplicates(subset = ['u1_u2'])
u.shape

In [ ]:
linksDF = pd.DataFrame()
linksDF['u1_u2'] = links_df[links_df['duplicate'] == True].drop_duplicates(subset = ['u1_u2'])['u1_u2']
linksDF ['user_1']= links_df[links_df['duplicate'] == True].drop_duplicates(subset = ['u1_u2'])['u_1']
linksDF['user_2']=links_df[links_df['duplicate'] == True].drop_duplicates(subset = ['u1_u2'])['u-2']
print(linksDF.shape)
linksDF.head()  

In [ ]:
links_final = u.merge(linksDF, how = 'left', on = ['u1_u2'] )
links_final.drop('weight_x', axis =1, inplace = True)
links_final.rename(columns = {'weight_y': 'weight'}, inplace = True)
print(links_final.shape)
links_final.head()

In [ ]:
links_final.to_csv('data_collection/data/linksFinal.csv')

In [275]:
links_final = pd.read_csv('data_collection/data/linksFinal.csv')

In [ ]:
def calculate_cosine(entry):
    user1 = entry["user1"][0]
    user2 = entry["user2"][0]
    value = float(entry["value"])/float(user1["totalvalue"])/float(user2["totalvalue"])
    db["link_merge"].update({"_id.user1": entry["_id"]["user1"], "_id.user2": entry["_id"]["user2"]}, {"$set": {"similarity": value}})
    return

def cosine_similarity():
    count = 0
    t0 = time()
    for link in db["link_merge"].find(no_cursor_timeout=True):
        if "similarity" not in entry:
            calculate_cosine(entry)
            count += 1
            if count%1000 == 0:
                print "count: ", count
                print("done in %0.3fs." % (time() - t0))
                t0 = time()

In [277]:
float(1)

1.0

In [272]:
''.join(s.split(':'))

'"hello"how'

In [267]:
s

'"hello":how:'

In [271]:
s = s.replace(' ', '')
s

'"hello":how:'

In [273]:
s= {'hello' : 'world'}